In [7]:
#IMPORTING SOME LIBRARY

import pandas as pd
import numpy as np
from scipy.signal import argrelextrema
from collections import defaultdict
import warnings
import time
import matplotlib.pyplot as plt
import matplotlib
from tqdm import tqdm
import json
import requests

from datetime import datetime, date, timedelta

In [8]:
#Config chart

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))
%config InlineBackend.figure_format = 'retina'

In [9]:
url = "https://api.binance.com/api/v3/klines?symbol=ETHUSDT&interval=1m"
r = requests.get(url)
r.json()


[[1619400180000,
  '2417.83000000',
  '2417.83000000',
  '2413.24000000',
  '2413.77000000',
  '398.49798000',
  1619400239999,
  '962639.17774130',
  794,
  '157.40708000',
  '380138.25099520',
  '0'],
 [1619400240000,
  '2413.81000000',
  '2417.81000000',
  '2411.84000000',
  '2416.00000000',
  '730.39512000',
  1619400299999,
  '1764066.33584700',
  990,
  '350.38525000',
  '846188.24747280',
  '0'],
 [1619400300000,
  '2416.01000000',
  '2419.97000000',
  '2412.37000000',
  '2414.41000000',
  '1088.61641000',
  1619400359999,
  '2630828.74655390',
  1116,
  '480.36678000',
  '1160872.32276590',
  '0'],
 [1619400360000,
  '2414.34000000',
  '2421.82000000',
  '2413.62000000',
  '2421.82000000',
  '809.63049000',
  1619400419999,
  '1957347.20506420',
  918,
  '488.25434000',
  '1180402.30825690',
  '0'],
 [1619400420000,
  '2421.67000000',
  '2426.92000000',
  '2415.38000000',
  '2416.27000000',
  '1417.62023000',
  1619400479999,
  '3434862.62915810',
  1760,
  '793.93721000',
  '1